# R-L delay tests

[L. Blackburn, Sep 2018; rewritten for Python 3.9, Dec 2022]

The purpose of this test is to check stability of R-L delay alignment in one antenna using a reference signal (either polarization) from another antenna. Outliers in R-L delay indicate a mis-fringe in the parallel and/or cross hand polarization product of the baselines being tested. The test is sensitive to delayed leakage in either antenna which can cause a spurious cross-hand fringe. It can also show misidentified singleband delay, e.g. from being outside the search window.

In [ ]:
# basic import and helper functions
import pandas as pd
from eat.io import hops, util
from eat.hops import util as hu
from eat.plots import util as pu
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import seaborn as sns

sns.reset_orig()
# sns.set_palette(sns.color_palette(sns.hls_palette(16, l=.6, s=.6)))
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# %config InlineBackend.figure_formats=['svg']

nb_stdout = sys.stdout # grab for later

def wide(w=8, h=3): plt.setp(plt.gcf(), figwidth=w, figheight=h); \
    plt.tight_layout()

def tightx(): plt.autoscale(enable=True, axis='x', tight=True)

def multline(xs, fun=plt.axvline):
    for x in xs: fun(x, alpha=0.25, ls='--', color='k')

def toiter(x):
    return(x if hasattr(x, '__iter__') else [x,])

# pd.options.display.float_format = '{:,.6f}'.format
from IPython.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))

In [ ]:
# define and load data
alistf = 'alist.v6'
datadir = os.environ['DATADIR']

a = util.noauto(hops.read_alist(os.path.join(datadir, alistf)))

# Pre-process alist dataframe
util.fix(a)
util.unwrap_mbd(a)
util.add_days(a)
util.add_delayerr(a)
util.add_path(a)
util.add_scanno(a)

In [ ]:
# data filters -- remove SMAR-SMAW baselines (only applicable to EHT2017 data)
thres = 7.0
a = a[(a.snr > thres) & ~a.baseline.isin({'RS', 'SR'})].copy()

In [ ]:
# Compute the boundaries between expt_nos
sorted_a = a.sort_values(['expt_no', 'scan_no'])
last_scans = sorted_a.groupby('expt_no')['scan_no'].max() # Find the 'max' scan_no for each expt_no
elines = (last_scans.iloc[:-1] + 0.5).to_numpy() # Drop the final expt_no and offset by 0.5

In [ ]:
# show all output in cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Get sorted list of unique sites from the baseline column
sites = sorted(set().union(*set(a.baseline)))

# Define cutoffs
lroffset_thres = 0.000200
lrstd_thres = 5.0

for site in sites:
    try:
        (p, stats) = hu.rl_segmented(a, site, restarts=hu.restarts)
        hu.rlplot(p, corrected=True)
        
        multline(elines)
        
        outliers = ((np.abs(p.LR_offset) > lroffset_thres) & (np.abs(p.LR_std) > lrstd_thres)
                    & ~(p.baseline.str.contains('L')
                    & (np.abs(np.abs(p.LR_offset) - 0.00145) < lroffset_thres)))
        if len(outliers) > 0:
            _ = plt.plot(p[outliers].scan_no, 1e3*p[outliers].LR_offset_wrap,
                         'ko', ms=8, mfc='none', mew=2, zorder=-100)
        _ = plt.title('R-L delay after subtracting mean value [%.0f MHz]' % (p.iloc[0].ref_freq))
        _ = plt.xlim(0, 1.05*plt.xlim()[1])
        wide(12, 5)
        plt.show()

        p.loc[outliers, "expt_no scan_id source timetag baseline ref_pol mbd_unwrap LR_offset LR_offset_wrap".split()]

    except Exception as e:
        print(f"Error processing site {site}: {e}\nMoving on to next site...")
        continue